## Librerias

In [27]:
import numpy as np
import pandas as pd

# stellargraph
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN

# sklearn
from sklearn import model_selection

#tensorflow
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

import pickle

## Abrimos los datos

In [28]:
#abrimos los datos
with open("./Datos/graphs.txt", "rb") as fp:   # Unpickling ../input/egg-new/26-0.2/graphs.txt
    graphs = pickle.load(fp)

with open("./Datos/labels.txt", "rb") as fp:   # Unpickling
    labels = pickle.load(fp)

# visualizamos como estan los datos importados
print(graphs[0].info())
print(labels[0])

StellarGraph: Undirected multigraph
 Nodes: 25, Edges: 52

 Node types:
  default: [25]
    Features: none
    Edge types: default-default->default

 Edge types:
    default-default->default: [52]
        Weights: range=[0.659464, 0.961213], mean=0.76453, std=0.0779707
        Features: none
0


## dataframe de los grafos

In [29]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)


,nodes,edges
count,906.0,906.0
mean,25.0,52.6
std,0.0,35.4
min,25.0,1.0
25%,25.0,25.0
50%,25.0,47.0
75%,25.0,73.0
max,25.0,223.0


## Dataframe de las labels

In [30]:

graph_labels = pd.get_dummies(labels, drop_first=True)



## Dividir en los splits

In [31]:
training_graphs, pred_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.90, test_size=None, stratify=graph_labels, random_state=6,
)

train_graphs, test_graphs = model_selection.train_test_split(
    training_graphs, train_size=0.85, test_size=None, stratify=training_graphs, random_state=6,
    
)

print('Test size:', len(pred_graphs.values))
print('Validation size:', len(test_graphs.values))
print('Training size:',len(train_graphs.values))

Test size: 91
Validation size: 123
Training size: 692


## Mini-batch con los grafos y las labels de cada set de entrenamiento

In [ ]:
gen = PaddedGraphGenerator(graphs=graphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=25,
    symmetric_normalization=True,
    weighted = True
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=25,
    symmetric_normalization=True,
    weighted = True
)

pred_gen = gen.flow(
    list(pred_graphs.index - 1),
    targets=pred_graphs.values,
    batch_size=1,
    symmetric_normalization=True,
    weighted = True
)

## Definición del modelo

In [32]:
def create_model(graphs, k, layer_sizes, activations1, activations2, dense_layers, dense_units, optimizer, conv, dropout_amount, dropout=None):
    generator = PaddedGraphGenerator(graphs=graphs)
    k = k  # the number of rows for the output tensor
    layer_sizes = layer_sizes

    dgcnn_model = DeepGraphCNN(
        layer_sizes= layer_sizes,
        activations= activations1,
        k=k,
        bias=False,
        generator=generator,
    )
    x_inp, x_out = dgcnn_model.in_out_tensors()
    
    if(conv):
        x_out = Conv1D(filters=16, kernel_size=sum(layer_sizes), strides=sum(layer_sizes))(x_out)
    
    x_out = Flatten()(x_out)


    for i in range(dense_layers):
        if((i % dropout_amount) == 0):
            if(dropout):
                x_out = Dropout(rate=dropout)(x_out)
                
        if(dense_units < 32):
            dense_units = 32
    
        x_out = Dense(units=dense_units, activation= activations2)(x_out)
        dense_units /= 2

    
    predictions = Dense(units=1, activation="sigmoid")(x_out)
    
    model = Model(inputs=x_inp, outputs=predictions)
    
    model.summary()
    return model

In [33]:
model = create_model(graphs, 25, [16,32], ['tanh', 'tanh'], 'relu', 1, 32, 'optimizers.Adagrad()', 0, 5, None)
model.compile(
    optimizer=optimizers.Adam(), loss=binary_crossentropy, metrics=["acc"],
)

# checkpoint
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

history = model.fit(
    train_gen, epochs=1000, verbose=1, validation_data=test_gen, shuffle=False
)

TypeError: __init__() missing 1 required positional argument: 'generator'